# Try psm format

In [10]:
import pandas as pd

psm_df = pd.read_csv("./dataset/TCGA/Proteomics/Downloads/TCGA_A2-A0SW_AO-A0JL_BH-A0BV_117C_W_BI_20131024_H-PM_f16.raw.cap.psm", sep="\t")

print(psm_df.columns)
print(psm_df.head())


Index(['FileName', 'ScanNum', 'QueryPrecursorMz', 'OriginalPrecursorMz',
       'PrecursorError(ppm)', 'QueryCharge', 'OriginalCharge',
       'PrecursorScanNum', 'PrecursorArea', 'PrecursorRelAb',
       'RTAtPrecursorHalfElution', 'PeptideSequence', 'AmbiguousMatch',
       'Protein', 'DeNovoScore', 'MSGFScore', 'Evalue', 'Qvalue', 'PepQvalue',
       'PrecursorPurity', 'FractionDecomposition', 'HCDEnergy', 'iTRAQ114',
       'iTRAQ115', 'iTRAQ116', 'iTRAQ117', 'iTRAQFlags', 'iTRAQTotalAb',
       'iTRAQFractionOfTotalAb'],
      dtype='object')
                                                    FileName   ScanNum  \
TCGA_A2-A0SW_AO-A0JL_BH-A0BV_117C_W_BI_20131024...       365  343.5114   
TCGA_A2-A0SW_AO-A0JL_BH-A0BV_117C_W_BI_20131024...      2129  347.9633   
TCGA_A2-A0SW_AO-A0JL_BH-A0BV_117C_W_BI_20131024...      2132  494.6088   
TCGA_A2-A0SW_AO-A0JL_BH-A0BV_117C_W_BI_20131024...      2144  391.5446   
TCGA_A2-A0SW_AO-A0JL_BH-A0BV_117C_W_BI_20131024...      2155  561.3084   

 

In [ ]:
# Apply filtering
filtered_psm_df = psm_df[
    (psm_df["PrecursorError(ppm)"] < 10) &
    (psm_df["Qvalue"] < 0.01) &
    (psm_df["PepQvalue"] < 0.01) &
    (psm_df["PrecursorPurity"] > 90) &
    (psm_df["AmbiguousMatch"] == 0)  # Keep only unambiguous matches
]

# Example: Grouping peptides by protein
peptide_to_protein = filtered_psm_df.groupby("Protein")["PeptideSequence"].apply(set)


# Group by Protein and compute median iTRAQ intensities
protein_expression_df = filtered_psm_df.groupby("Protein")[["iTRAQ114", "iTRAQ115", "iTRAQ116", "iTRAQ117"]].median()


# Try 

In [29]:
import pandas as pd
import requests
import os
from tqdm import tqdm  # Progress bar

# Load the case-to-file mapping data
matched_df = pd.read_csv("dataset/processed/1. Data_preparation-matching_ID/Proteome/matched_samples_with_aliquot.csv")
gdc_df = pd.read_csv("dataset/CPTAC/RNA-Seq/gdc_sample_sheet.2025-02-14.tsv", sep="\t")

# Ensure Case IDs are strings to prevent type mismatch
matched_samples = set(matched_df["Case Submitter ID"].astype(str))

# Create a mapping of Case ID → (File ID, File Name)
case_to_file = {
    row["Case ID"]: (row["File ID"], row["File Name"])
    for _, row in gdc_df.iterrows() if row["Case ID"] in matched_samples
}

# Convert to DataFrame for easier visualization
case_to_file_df = pd.DataFrame(
    [(case, file_id, file_name) for case, (file_id, file_name) in case_to_file.items()],
    columns=["Case ID", "File ID", "File Name"]
)

# Create a directory to store downloaded files
download_dir = "dataset/processed/1. Data_preparation-matching_ID/RNA-seq"
os.makedirs(download_dir, exist_ok=True)

# GDC API endpoint
gdc_api_url = "https://api.gdc.cancer.gov/data/{}"

# 🔹 2️⃣ Batch download files with tqdm progress bar
for idx, row in tqdm(case_to_file_df.iterrows(), total=len(case_to_file_df), desc="📥 Download Progress", unit="file"):
    file_id = row["File ID"]
    file_name = row["File Name"]
    file_path = os.path.join(download_dir, file_name)

    # Check if the file already exists
    if os.path.exists(file_path):
        print(f"✅ {file_name} already exists. Skipping download.")
        continue  # Skip to the next file

📥 Download Progress: 100%|██████████| 121/121 [00:00<00:00, 16208.19file/s]

✅ 29ec889a-6fef-40d9-9a6c-570e61c25d0b.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ b7e3fbe6-1ece-4bbb-9d50-222daafe0e34.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ a9972ef0-7002-463a-bc65-c59c6d50ac59.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ b29b00ab-501a-4797-babc-6628eba5f501.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ edeeefbe-8d5f-46a2-a75c-549688de51ae.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ 8dabfed6-1eb1-4915-8203-66093d2cf43f.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ 238ce745-a5f9-4f2e-a9c6-4e353d72bbeb.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ 136e782c-cfa4-450a-8ed7-55a1d610c12c.rna_seq.augmented_star_gene_counts.tsv already exists. Skipping download.
✅ 085415a5-37c0-456e-89aa-2c7c4e1230ab.rna_seq.augmented_star_gene_counts.tsv already exists. Sk

In [14]:
en = cptac.cptac.Brca()
en.list_data_sources()


,Data type,Available sources
0,CNV,"[bcm, washu]"
1,miRNA,[bcm]
2,phosphoproteomics,"[bcm, umich]"
3,proteomics,"[bcm, umich]"
4,transcriptomics,"[bcm, broad, washu]"
5,ancestry_prediction,[harmonized]
6,somatic_mutation,"[harmonized, washu]"
7,clinical,[mssm]
8,follow-up,[mssm]
9,medical_history,[mssm]


In [15]:
# there is also a get function for each datatype, this is the same as get_dataframe 
# but does not need the datatype argument. This function call below is equivalent to 
# the one above
proteomics = en.get_proteomics('umich')

samples = proteomics.index
proteins = proteomics.columns
print("Samples:",samples[0:20].tolist()) #the first twenty samples
print("Proteins:",proteins[0:20].tolist()) #the first twenty proteins

Samples: ['01BR001', '01BR008', '01BR009', '01BR010', '01BR015', '01BR017', '01BR018', '01BR020', '01BR023', '01BR025', '01BR026', '01BR027', '01BR030', '01BR031', '01BR032', '01BR033', '01BR040', '01BR042', '01BR043', '03BR002']
Proteins: [('ARF5', 'ENSP00000000233.5'), ('M6PR', 'ENSP00000000412.3'), ('ESRRA', 'ENSP00000000442.6'), ('FKBP4', 'ENSP00000001008.4'), ('NDUFAF7', 'ENSP00000002125.4'), ('FUCA2', 'ENSP00000002165.5'), ('DBNDD1', 'ENSP00000002501.6'), ('SEMA3F', 'ENSP00000002829.3'), ('CFTR', 'ENSP00000003084.6'), ('CYP51A1', 'ENSP00000003100.8'), ('USP28', 'ENSP00000003302.4'), ('TMEM176A', 'ENSP00000004103.3'), ('SLC7A2', 'ENSP00000004531.10'), ('HSPB6', 'ENSP00000004982.3'), ('ZNF195', 'ENSP00000005082.9'), ('PDK4', 'ENSP00000005178.5'), ('RALA', 'ENSP00000005257.2'), ('BAIAP2L1', 'ENSP00000005260.8'), ('TMEM132A', 'ENSP00000005286.4'), ('DVL2', 'ENSP00000005340.4')]


In [16]:
proteomics

Name,ARF5,M6PR,ESRRA,FKBP4,NDUFAF7,FUCA2,DBNDD1,SEMA3F,CFTR,CYP51A1,...,DDHD1,WIZ,GBF1,APOA5,WIZ,LDB1,WIZ,RFX7,SWSAP1,SVIL
Database_ID,ENSP00000000233.5,ENSP00000000412.3,ENSP00000000442.6,ENSP00000001008.4,ENSP00000002125.4,ENSP00000002165.5,ENSP00000002501.6,ENSP00000002829.3,ENSP00000003084.6,ENSP00000003100.8,...,ENSP00000500986.2,ENSP00000500993.1,ENSP00000501064.1,ENSP00000501141.1,ENSP00000501256.3,ENSP00000501277.1,ENSP00000501300.1,ENSP00000501317.1,ENSP00000501355.1,ENSP00000501521.1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01BR001,0.012367,-0.945999,NaN,-0.478170,1.135840,-0.512706,0.750335,-0.274824,NaN,-0.278244,...,-0.649127,-0.580869,-0.226667,NaN,NaN,-0.676185,-0.068202,-0.078207,-0.328420,NaN
01BR008,-0.514386,0.462307,0.230124,-0.555968,0.491366,-0.656034,-1.220890,-0.369282,-1.036441,-0.059327,...,0.632221,NaN,0.032873,NaN,NaN,-0.015459,0.227424,0.325643,-0.606240,NaN
01BR009,-0.210782,-0.085055,0.380296,-0.389491,1.255391,-0.608007,-0.231318,0.092870,-1.505195,0.206595,...,0.450818,NaN,-0.341503,NaN,NaN,-0.220239,0.125092,0.365397,-0.167392,NaN
01BR010,0.105457,0.351335,-0.322798,-0.821610,0.241406,-0.500140,-0.137824,0.113791,NaN,0.498314,...,-0.423470,NaN,0.360900,NaN,NaN,-0.451556,-0.098897,0.208643,-0.729096,0.670307
01BR015,-0.509298,-0.874164,NaN,-0.113804,-0.131347,-0.412813,0.262210,0.042333,NaN,-0.657666,...,0.406016,-0.493869,-0.192847,NaN,NaN,0.083639,0.966976,-0.012664,0.081968,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21BR010,0.528298,-0.127929,-0.497360,-0.151022,0.082288,0.447267,0.151024,0.220194,0.516739,-0.230357,...,-0.172151,0.636608,0.267400,NaN,-0.09507,-0.017522,-0.220463,-0.067717,-0.311446,0.602422
22BR005,-0.549542,0.134236,NaN,0.580773,-0.080663,-0.056509,-0.148632,0.260986,NaN,-0.348578,...,0.791937,NaN,0.171712,NaN,NaN,0.083980,-0.200083,0.155198,NaN,0.456801


In [22]:

transcriptomics = en.get_transcriptomics('bcm')
transcriptomics.index


Index(['01BR001', '01BR008', '01BR009', '01BR010', '01BR015', '01BR017',
       '01BR018', '01BR020', '01BR023', '01BR025',
       ...
       '20BR005', '20BR006', '20BR007', '20BR008', '21BR001', '21BR002',
       '21BR010', '22BR005', '22BR006', '604'],
      dtype='object', name='Patient_ID', length=133)

In [26]:
import pandas as pd

# 2️⃣ 讀取 GDC Sample Sheet
gdc_df = pd.read_csv("./dataset/TCGA/RNA-Seq/gdc_sample_sheet.2025-02-14.tsv",sep="\t")

# 3️⃣ 取得 Case ID 集合
transcriptomics_cases = set(transcriptomics.index)  # transcriptomics 的 Case ID
gdc_cases = set(gdc_df["Case ID"])  # GDC Sample Sheet 的 Case ID

# 4️⃣ 找出差異
only_in_transcriptomics = transcriptomics_cases - gdc_cases  # 只在 transcriptomics 內
only_in_gdc = gdc_cases - transcriptomics_cases  # 只在 GDC Sample Sheet 內

# 5️⃣ 印出結果
print(f"✅ transcriptomics.index 有，但 GDC Sample Sheet 沒有的 Case ID:\n{only_in_transcriptomics}\n")
print(f"✅ GDC Sample Sheet 有，但 transcriptomics.index 沒有的 Case ID:\n{only_in_gdc}\n")

# 6️⃣ 存成 CSV 方便查看
pd.DataFrame({"Missing in GDC": list(only_in_transcriptomics)}).to_csv("missing_in_gdc.csv", index=False)
pd.DataFrame({"Missing in transcriptomics": list(only_in_gdc)}).to_csv("missing_in_transcriptomics.csv", index=False)
# 取得兩者皆有的 Case ID（交集）
common_cases = transcriptomics_cases & gdc_cases

# 印出結果
print(f"✅ 兩者皆有的 Case ID:\n{common_cases}\n")

# 存成 CSV 方便查看
pd.DataFrame({"Common Case IDs": list(common_cases)}).to_csv("common_case_ids.csv", index=False)


✅ transcriptomics.index 有，但 GDC Sample Sheet 沒有的 Case ID:
set()

✅ GDC Sample Sheet 有，但 transcriptomics.index 沒有的 Case ID:
set()

✅ 兩者皆有的 Case ID:
{'06BR014', '11BR044', '09BR001', '01BR033', '05BR045', '05BR042', '13BR009', '11BR032', '18BR009', '05BR029', '05BR044', '11BR053', '18BR003', '14BR005', '03BR006', '11BR036', '11BR016', '11BR012', '01BR032', '05BR055', '11BR038', '03BR010', '11BR054', '11BR027', '06BR005', '15BR003', '05BR009', '100004012', '05BR043', '100003304', '11BR024', '11BR030', '01BR025', '05BR052', '09BR004', '01BR030', '03BR012', '05BR058', '11BR013', '20BR007', '11BR059', '01BR044', '09BR007', '11BR049', '11BR056', '03BR004', '11BR020', '100004028', '1488', '11BR050', '01BR009', '11BR072', '11BR080', '06BR006', '18BR010', '11BR051', '20BR005', '14BR008', '11BR074', '11BR004', '05BR003', '16BR012', '22BR005', '11BR028', '01BR042', '20BR006', '18BR006', '18BR004', '18BR017', '05BR001', '11BR006', '11BR014', '11BR040', '11BR025', '05BR031', '01BR023', '01BR020', '6

In [27]:
len(common_cases)

133